In [37]:
import pandas as pd
import numpy as np
import decimal as D

In [38]:
df = pd.read_csv("2021-02-06-SettlementNotice - Visa VSS 910.csv", sep=";", parse_dates=["ReportDate"],  thousands=".", decimal=",", usecols=["ReportDate", "BusinessTransactionCycle", "GrossAmount", "DefermentDays"], dtype={"GrossAmount": np.float64})

In [39]:
df.DefermentDays.unique()

array(['   ', '002', '027'], dtype=object)

In [40]:
df.insert(4, "eventType", "")

In [41]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Originals" and row["DefermentDays"] == "027":
        df.loc[index, "eventType"] = "credit"
    elif row["BusinessTransactionCycle"] == "Originals" and row["DefermentDays"] == "   ":
        df.loc[index, "eventType"] = "debit"
    

In [42]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Dispute Financial":
        df.loc[index, "eventType"] = "chargeback"
    elif row["BusinessTransactionCycle"] == "Dispute Response Financial":
        df.loc[index, "eventType"] = "secondPresentment"

In [43]:
df["ReportDate"] = pd.to_datetime(df["ReportDate"])

In [44]:
df["BusinessTransactionCycle"] = df["BusinessTransactionCycle"].astype("str")

In [45]:
df.dtypes

ReportDate                  datetime64[ns]
BusinessTransactionCycle            object
GrossAmount                        float64
DefermentDays                       object
eventType                           object
dtype: object

In [46]:
df.head()

,ReportDate,BusinessTransactionCycle,GrossAmount,DefermentDays,eventType
0,2021-02-05,The transaction does not have a cycle.,433.85,,
1,2021-02-05,The transaction does not have a cycle.,5138.17,002,
2,2021-02-05,Dispute Financial,74.79,,chargeback
3,2021-02-05,Originals,71091.71,,debit
4,2021-02-05,Originals,1776864.18,027,credit


In [47]:
df = df.drop(columns=["BusinessTransactionCycle","DefermentDays"])

In [50]:
df

,ReportDate,GrossAmount,eventType
0,2021-02-05,4.338500e+02,
1,2021-02-05,5.138170e+03,
2,2021-02-05,7.479000e+01,chargeback
3,2021-02-05,7.109171e+04,debit
4,2021-02-05,1.776864e+06,credit
5,2021-02-05,5.050209e+05,chargeback
6,2021-02-05,1.686062e+04,secondPresentment
7,2021-02-05,3.204443e+04,
8,2021-02-05,8.404387e+07,debit
9,2021-02-05,1.143416e+08,credit


In [51]:
df = df[['ReportDate', 'eventType', 'GrossAmount']]

In [52]:
df

,ReportDate,eventType,GrossAmount
0,2021-02-05,,4.338500e+02
1,2021-02-05,,5.138170e+03
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
7,2021-02-05,,3.204443e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08


In [53]:
df.rename(columns={"ReportDate": "confirmedDate",
                  "GrossAmount": "amountInBRL"})

,confirmedDate,eventType,amountInBRL
0,2021-02-05,,4.338500e+02
1,2021-02-05,,5.138170e+03
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
7,2021-02-05,,3.204443e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08


In [55]:
#df.drop(df[df['eventType']=""].index, inplace = True) 

SyntaxError: invalid syntax (<ipython-input-55-43e8745d55d0>, line 1)

In [58]:
rows_to_delete = []

In [59]:
for index, row in df.iterrows():
    if row["eventType"] == "":
        rows_to_delete.append(index)

In [60]:
rows_to_delete

[0, 1, 7, 10, 11, 12, 17]

In [61]:
df = df.drop(rows_to_delete)

In [62]:
df

,ReportDate,eventType,GrossAmount
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08
13,2021-02-06,debit,8.348275e+04
14,2021-02-06,credit,1.832042e+06
15,2021-02-06,chargeback,4.963857e+05
